<a href="https://colab.research.google.com/github/obeabi/AirlineSentiment/blob/main/ShortTrades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
!pip install --upgrade yfinance
!pip install  --upgrade pandas_ta
!pip install ta pandas_ta

In [180]:
import yfinance as yf
print(yf.__version__)
import pandas as pd
#import pandas_ta as ta
import numpy as np
from datetime import datetime
import time
import ta
print("Libraries Installed!")

0.2.65
Libraries Installed!


In [181]:
# List of ETFs to analyze

df_raw = pd.read_csv('short_list.csv')
etfs = df_raw['Asset'].to_list()
print(etfs)

print(len(etfs))

['WALD', 'THS', 'HNST', 'BRCC', 'BYND', 'EPC', 'SMPL', 'GNE', 'UTL', 'IPAR', 'WEST', 'ZVIA', 'UVV', 'COCO', 'CENT', 'AVA', 'FIZZ']
17


In [182]:
# Filter ETFs or stocks for liquidity
def filter_by_liquidity(etf_df, ticker_col="Asset", min_dollar_vol=1e6, lookback_days=30):
    liquid_etfs = []

    for ticker in etf_df[ticker_col]:
        try:
            # Fetch daily historical data
            data = yf.download(ticker, period=f"{lookback_days*2}d", interval="1d", auto_adjust=True)

            if data.empty:
                continue

            # Calculate dollar volume (Close × Volume)
            data["dollar_volume"] = data["Close"] * data["Volume"]

            # Calculate rolling average over lookback_days
            avg_dollar_volume = data["dollar_volume"].rolling(window=lookback_days).mean().iloc[-1]

            # Check liquidity condition
            if avg_dollar_volume >= min_dollar_vol:
                liquid_etfs.append(ticker)

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")

    # Return filtered DataFrame
    return etf_df[etf_df[ticker_col].isin(liquid_etfs)]

# Example usage
df = pd.DataFrame({"Assets": etfs})
liquid_df = filter_by_liquidity(df, ticker_col="Assets")
df_o = df_raw[df_raw['Asset'].isin(liquid_df['Assets'])]
etfs = df_o['Asset'].to_list()
print("")
print(etfs)
print(len(etfs))



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


['THS', 'HNST', 'BRCC', 'BYND', 'EPC', 'SMPL', 'GNE', 'UTL', 'IPAR', 'WEST', 'ZVIA', 'UVV', 'COCO', 'CENT', 'AVA', 'FIZZ']
16


In [183]:
# Function to fetch historical weekly data

def get_monthly_data(ticker):
  try:
      df = yf.download(ticker, period="10y", interval="1mo",auto_adjust=True)
      df['10_month_SMA'] = df['Close'].rolling(window=10).mean()
      df['SMA_Slope'] = df['10_month_SMA'].diff()/10
      # Calculate MACD and Signal Line
      df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
      df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
      # Compute MACD Line
      df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
      # Compute Signal Line (9-day EMA of MACD Line)
      df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
      df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']

      return df
  except Exception as e:
      print("There is an error getting monthly data", e)

def get_weekly_data(ticker):
  try:
      df = yf.download(ticker, period="2y", interval="1wk",auto_adjust=True)
      df['10_week_SMA'] = df['Close'].rolling(window=10).mean()
      df['20_week_SMA'] = df['Close'].rolling(window=20).mean()
      df['50_week_SMA'] = df['Close'].rolling(window=50).mean()
      df['30_week_SMA'] = df['Close'].rolling(window=30).mean()
      df['SMA_Slope'] = df['20_week_SMA'].diff()/20
      df['10_EMA'] = df['Close'].ewm(span=10, adjust=False).mean()
      df['20_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()

      df['RSI'] = compute_rsi(df['Close'])
      df['ATR'] = compute_atr(df, 14)
      df['OBV'] = compute_obv(df)
      df['OBV_Slope'] = df['OBV'].diff()
      df['20_week_avg_volume'] = df['Volume'].rolling(window=20).mean()
      df['ma'] = calculate_ma(df['RSI'])
      df['upper_band'], df['lower_band'] = calculate_bollinger_bands(df['RSI'])

      # Calculate MACD and Signal Line
      df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
      df['21_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
      df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
      # Compute MACD Line
      df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
      # Compute Signal Line (9-day EMA of MACD Line)
      df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
      df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']

      # Compute raw EFI
      df['EFI'] = (df['Close'].diff()) * df['Volume']
      # Compute EMA of EFI
      df['EFI_EMA'] = df['EFI'].ewm(span=13, adjust=False).mean()
      # Determine if EFI_EMA is rising or falling
      df['EFI_EMA_Trend'] = df['EFI_EMA'].diff().apply(lambda x: 'Rising' if x > 0 else 'Falling')
      # Calculate ADX, +DMI and -DMI
      high = df['High']
      low = df['Low']
      close = df['Close']
      # Calculate directional movements
      up_move = high.diff()
      down_move = -low.diff()
      plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
      minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
      # Calculate True Range (TR)
      tr1 = high - low
      tr2 = (high - close.shift()).abs()
      tr3 = (low - close.shift()).abs()
      tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
      # Smooth TR, +DM, and -DM using Wilder’s smoothing
      atr = tr.rolling(window=14).sum()
      plus_dm_series = pd.Series(plus_dm.ravel(), index=df.index).astype(float)
      minus_dm_series = pd.Series(minus_dm.ravel(), index=df.index).astype(float)
      plus_dm_smoothed = plus_dm_series.rolling(window=14).sum()
      minus_dm_smoothed = minus_dm_series.rolling(window=14).sum()
      # Directional Indicators
      plus_di = 100 * (plus_dm_smoothed / atr)
      minus_di = 100 * (minus_dm_smoothed / atr)
      # DX and ADX
      dx = 100 * (np.abs(plus_di - minus_di) / (plus_di + minus_di))
      adx = dx.rolling(window=14).mean()
      # Add results to original DataFrame
      df['+DI'] = plus_di
      df['-DI'] = minus_di
      df['ADX'] = adx
      df['di_flag'] = df['+DI'] < df['-DI']
      df['di_flag'] = df['di_flag'].astype(int)
      df['adx_indicator'] = np.where(df['ADX'] > 25, 1, 0)
      df['adx_signal'] = df['adx_indicator'] * df['di_flag']

      # --- Overhead Resistance Filter ---
      recent_52_weeks = df[-52:]
      min_close_52w = recent_52_weeks['Close'].min()
      last_close = df['Close'].iloc[-1].iloc[0]
      # --- Above 52 weeks Low ---
      df['above_52w_low'] = last_close > min_close_52w
      df['below_52w_low'] = last_close < min_close_52w

      return df
  except Exception as e:
      print("There is an error getting weekly data", e)

def is_macd_bullish(df):
    """
    Determines if there is a bullish signal on the MACD indicator.
    A bullish signal occurs when the MACD line is above the signal line.

    Parameters:
    - df: dataframe. Must have columns 'MACD_Line' and 'Signal_Line'.
    Returns:
    - bool: True if a bullish signal is detected, otherwise False.
    """
    try:
      if df.empty:
        return False
      # Calculate MACD histogram if not already present

      curr = df.iloc[-3:]
      macd_crossover = curr['MACD_Line'].iloc[-1] < curr['Signal_Line'].iloc[-1]
      below_zero_line = curr['MACD_Line'].iloc[-1] < 0


      return macd_crossover or below_zero_line

    except Exception as e:
      print("Something went wrong while computing the MACD", e)

def calculate_bbw(df, window=20):
    """ Calculates Bollinger Bands Width"""
    sma = df['Close'].rolling(window).mean()
    std = df['Close'].rolling(window).std()
    upper = sma + 2*std
    lower = sma - 2*std
    bbw = (upper - lower) / sma
    return bbw

def calculate_vwap(df):
  """ Calculates vwap"""
  try:
    Typical_Price = (df['Close'].values + df['High'].values + df['Low'].values) / 3
    TPV = Typical_Price * df['Volume'].values
    vwap = TPV.cumsum() / df['Volume'].values.cumsum()
    return vwap
  except Exception as e:
    print("Something went wrong while computing the VWAP:", e)
    return None

# Function to compute RSI
def compute_rsi(series, period=10):
  try:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
  except Exception as e:
        print("Something went wrong while computing the RSI:", e)
        return None

def calculate_ma(data, length=10, ma_type="WMA"):
    if ma_type == "SMA":
        return data.rolling(window=length).mean()
    elif ma_type == "EMA":
        return data.ewm(span=length, adjust=False).mean()
    elif ma_type == "WMA":
        weights = np.arange(1, length+1)
        return data.rolling(length).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
    elif ma_type == "VWMA":
        return ta.volume_weighted_average_price(data, length)

def calculate_bollinger_bands(data, length=10, std_dev=2.0):
    sma = data.rolling(window=length).mean()
    std_dev = data.rolling(window=length).std()
    upper_band = sma + (std_dev * std_dev)
    lower_band = sma - (std_dev * std_dev)
    return upper_band, lower_band

# Function to compute ATR (Average True Range)
def compute_atr(df, period=10):
  try:
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df['TR'].rolling(window=period).mean()
  except Exception as e:
      print("Something went wrong whilecomputing the ATR", e)


# Function to compute On-Balance Volume (OBV)
def compute_obv(df):
  try:
    # Calculate daily price change: 1 if price is up, -1 if down, 0 if unchanged
    price_change = df['Close'].diff()

    # Use price change to decide whether to add or subtract volume
    obv = (price_change > 0).astype(int) * df['Volume']  # Volume when price goes up
    obv -= (price_change < 0).astype(int) * df['Volume']  # Volume when price goes down

    # We accumulate the OBV by taking the cumulative sum of the volume changes
    obv = obv.cumsum()

    return obv
  except Exception as e:
      print("Something went wrong while computing the OBV", e)


# Function to calculate risk-reward ratio
def calculate_risk_reward(df):
  try:
    if df.empty or len(df) < 20:  # Ensure there are enough data points
        return np.nan

    latest_price = df['Close'].iloc[-1].iloc[0]

    # Use the ATR for setting support level
    atr = df['ATR'].iloc[-1]  # Latest ATR value
    price_ema = df['21_day_EMA'].iloc[-1]
    price_8ema = df['8_day_EMA'].iloc[-1]
    atr_multiple = 1.5 # You can adjust this multiplier based on your strategy

    # Calculate the support level using the ATR
    trailing = atr * atr_multiple
    support_level = price_ema + trailing
    ema_level = price_8ema

    return support_level, latest_price, trailing, ema_level
  except Exception as e:
      print("Something went wrong while computing the reward-risk ratio", e)

# Function to fetch daily data
def get_daily_data(ticker):
    df = yf.download(ticker, period="1y", interval="1d",auto_adjust=True)
    df['20_day_SMA'] = df['Close'].rolling(window=20).mean()
    df['50_day_avg_volume'] = df['Volume'].rolling(window=50).mean()
    df['8_day_EMA'] = df['Close'].ewm(span=8, adjust=False).mean()
    df['5_day_EMA'] = df['Close'].ewm(span=5, adjust=False).mean()
    df['21_day_EMA'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['26_day_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['50_day_SMA'] = df['Close'].rolling(window=50).mean()
    df['100_day_SMA'] = df['Close'].rolling(window=100).mean()
    df['200_day_SMA'] = df['Close'].rolling(window=200).mean()
    df['SMA_Slope_20'] = df['20_day_SMA'].diff()/20
    df['SMA_Slope_50'] = df['50_day_SMA'].diff()/50
    df['SMA_Slope_100'] = df['100_day_SMA'].diff()/100
    df['RSI'] = compute_rsi(df['Close'],period=10)
    df['ATR'] = compute_atr(df, 10)
    df["8EMA_minus_ATR"] = df["8_day_EMA"] - (1* df["ATR"])
    df['ma'] = calculate_ma(df['RSI'])
    df['BBW'] = calculate_bbw(df)
    df['upper_band'], df['lower_band'] = calculate_bollinger_bands(df['RSI'])
    # Calculate MACD and Signal Line
    df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    # Compute MACD Line
    df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
    # Compute Signal Line (9-day EMA of MACD Line)
    df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
    df['VWAP'] = calculate_vwap(df)
    # Compute raw EFI
    df['EFI'] = (df['Close'].diff()) * df['Volume']
    # Compute EMA of EFI
    df['EFI_EMA'] = df['EFI'].ewm(span=13, adjust=False).mean()
    # Determine if EFI_EMA is rising or falling
    df['EFI_EMA_Trend'] = df['EFI_EMA'].diff().apply(lambda x: 'Rising' if x > 0 else 'Falling')
    # Calculate ADX, +DMI and -DMI
    high = df['High']
    low = df['Low']
    close = df['Close']
    # Calculate directional movements
    up_move = high.diff()
    down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    # Calculate True Range (TR)
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    # Smooth TR, +DM, and -DM using Wilder’s smoothing
    atr = tr.rolling(window=14).sum()
    plus_dm_series = pd.Series(plus_dm.ravel(), index=df.index).astype(float)
    minus_dm_series = pd.Series(minus_dm.ravel(), index=df.index).astype(float)
    plus_dm_smoothed = plus_dm_series.rolling(window=14).sum()
    minus_dm_smoothed = minus_dm_series.rolling(window=14).sum()
    # Directional Indicators
    plus_di = 100 * (plus_dm_smoothed / atr)
    minus_di = 100 * (minus_dm_smoothed / atr)
    # DX and ADX
    dx = 100 * (np.abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window=14).mean()
    # Add results to original DataFrame
    df['+DI'] = plus_di
    df['-DI'] = minus_di
    df['ADX'] = adx
    df['di_flag'] = df['+DI'] < df['-DI']
    df['di_flag'] = df['di_flag'].astype(int)
    df['adx_indicator'] = np.where(df['ADX'] > 25, 1, 0)
    df['adx_signal'] = df['adx_indicator'] * df['di_flag']
    return df

# Function to fetch hourly data
def get_1hr_data(ticker):
    df = yf.download(ticker, interval='1h', period='1y',auto_adjust=True)

    df['20_hr_SMA'] = df['Close'].rolling(window=20).mean()
    df['50_hr_SMA'] = df['Close'].rolling(window=50).mean()
    df['SMA_Slope_20'] = df['20_hr_SMA'].diff()/20
    df['SMA_Slope_50'] = df['50_hr_SMA'].diff()/50
    # Calculate MACD and Signal Line
    df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    # Compute MACD Line
    df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
    # Compute Signal Line (9-day EMA of MACD Line)
    df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
    return df

# Function to fetch 15mins data to refine entry
def get_15min_data(ticker):
    df = yf.download(ticker, interval='15m', period='60d',auto_adjust=True)

    df['30_min_SMA'] = df['Close'].rolling(window=30).mean()
    df['50_min_SMA'] = df['Close'].rolling(window=50).mean()
    df['SMA_Slope'] = df['50_min_SMA'].diff()/50
    # Calculate MACD and Signal Line
    df['12_EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26_EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    # Compute MACD Line
    df['MACD_Line'] = df['12_EMA'] - df['26_EMA']
    # Compute Signal Line (9-day EMA of MACD Line)
    df['Signal_Line'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['Signal_Line']
    return df

# Function to check monthly trend
def is_monthly_trend_bullish(df):
    if df.empty:
        return False

    sma_slope = df['SMA_Slope'].iloc[-1]< 0
    latest_price = df['Close'].iloc[-1].iloc[0]
    latest_sma = df['10_month_SMA'].iloc[-1]
    macd_bullish_signal = is_macd_bullish(df)
    below_10_month_SMA = latest_price < latest_sma
    return below_10_month_SMA and macd_bullish_signal and sma_slope


# Function to check weekly trend
def is_weekly_trend_bullish(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    latest_sma = df['10_week_SMA'].iloc[-1]
    latest_30sma = df['30_week_SMA'].iloc[-1]
    below_10_week_SMA = latest_price < latest_sma
    below_30_week_SMA = latest_price < latest_30sma
    sma_slope = df['SMA_Slope'].iloc[-1]< 0
    obv_slope = df['OBV_Slope'].iloc[-1]< 0
    macd_bullish_signal = is_macd_bullish(df)
    elderforce_trend_ok = df['EFI_EMA_Trend'].iloc[-1] == 'Falling'
    adx_ok = df['adx_signal'].iloc[-1] == 1
    volume_ok = df['Volume'].iloc[-1] > df['20_week_avg_volume'].iloc[-1] # Institutional interest
    volume_ok = volume_ok.iloc[0]
    # Calculate the OBV Moving Average
    df['OBV_EMA'] = df['OBV'].ewm(span=10, adjust=False).mean()
    # OBV trending up if current OBV is above the 20-period EMA
    obv_trending_down = df['OBV'].iloc[-1] < df['OBV_EMA'].iloc[-1]
    trend_ok = below_10_week_SMA and below_30_week_SMA and sma_slope and adx_ok
    rsi_ok = df['RSI'].iloc[-1] < 50 # Not  oversold
    below_52w_low = df['below_52w_low'].iloc[-1]
    time.sleep(2)  # Add a delay of 1 second between requests

    return  trend_ok and (volume_ok or obv_trending_down or obv_slope ) and macd_bullish_signal  \
             #and below_52w_low #elderforce_trend_ok


# Function to check daily entry signal
def is_daily_entry_signal(df2):
    if df2.empty:
        return False

    # Apply BBW filter (must be > median)
    median_bbw = df2['BBW'].rolling(20).median()
    #df2['BBW_Filter'] = df2['BBW'] < median_bbw
    # Filter to generate signal
    #df = df2[df2['BBW_Filter']].copy()
    df = df2.copy()
    latest_price = df['Close'].iloc[-1].iloc[0]
    sma_slope_20 = df['SMA_Slope_20'].iloc[-1]< 0
    sma_slope_50 = df['SMA_Slope_50'].iloc[-1]< 0
    sma_slope_100 = df['SMA_Slope_100'].iloc[-1]< 0
    prev_price = df['Close'].iloc[-2].iloc[0]
    latest_20sma = df['20_day_SMA'].iloc[-1]
    latest_50sma = df['50_day_SMA'].iloc[-1]
    latest_100sma = df['100_day_SMA'].iloc[-1]
    latest_200sma = df['200_day_SMA'].iloc[-1]
    below_20sma = latest_price < latest_20sma
    below_50sma = latest_price < latest_50sma
    below_100sma = latest_price < latest_100sma
    below_200sma = latest_price < latest_200sma
    is_20sma_below_50sma = latest_20sma and latest_50sma
    is_50sma_below_100sma = latest_50sma and latest_100sma
    is_100sma_below_200sma = latest_100sma and latest_200sma
    latest_rsi = df['RSI'].iloc[-1]
    macd_bullish_signal = is_macd_bullish(df)
    vwap_price = df['VWAP'].iloc[-1]
    elderforce_trend_ok = df['EFI_EMA_Trend'].iloc[-1] == 'Falling'
    elderforce_ema_ok = df['EFI_EMA'].iloc[-1] < 0
    adx_ok = df['adx_signal'].iloc[-1] == 1
    slopes_ok = sma_slope_20 and sma_slope_50 and sma_slope_100
    moving_averages_ok = below_20sma and below_50sma and below_100sma and \
                         is_20sma_below_50sma and is_50sma_below_100sma and is_100sma_below_200sma


    # Look for a breakout above 20-day SMA & RSI > 50
    return (moving_averages_ok and slopes_ok) and  \
              adx_ok and macd_bullish_signal and (latest_price < vwap_price)
              #and elderforce_trend_ok and elderforce_ema_ok

# Check entry conditions
def check_entry_conditions(tickers):
    results = []
    for ticker in tickers:
      df = get_daily_data(ticker)
      latest_price = df['Close'].iloc[-1].iloc[0]
      prev_price = df['Close'].iloc[-2].iloc[0]
      latest_sma = df['50_day_SMA'].iloc[-1]
      latest_rsi = df['RSI'].iloc[-1]
      latest_price_5ema =df['5_day_EMA'].iloc[-1]
      latest_price_8ema =df['8_day_EMA'].iloc[-1]
      latest_price_21ema =df['21_day_EMA'].iloc[-1]
      price_threshold_ATR = df['8EMA_minus_ATR'].iloc[-1]
      above_price_threshold_ATR = latest_price >= price_threshold_ATR
      below_price_threshold_ATR = latest_price < price_threshold_ATR

      df_entry         = get_1hr_data(ticker)
      df_entry_refined = get_15min_data(ticker)

      latest_priceh_50sma  = df_entry['50_hr_SMA'].iloc[-1]
      latest_priceh_20sma  = df_entry['20_hr_SMA'].iloc[-1]
      latest_priceh        = df_entry['Close'].iloc[-1].iloc[0]
      sma_slope_20hr       = df_entry['SMA_Slope_20'].iloc[-1]< 0
      sma_slope_50hr       = df_entry['SMA_Slope_50'].iloc[-1]< 0



      latest_pricem_30sma  = df_entry_refined['30_min_SMA'].iloc[-1]
      latest_pricem        = df_entry_refined['Close'].iloc[-1].iloc[0]
      sma_slope_m          = df_entry_refined['SMA_Slope'].iloc[-1]< 0



      macd_bullish_signal_1hr   = is_macd_bullish(df_entry)
      #macd_bullish_signal_15min = is_macd_bullish(df_entry_refined)


      refined_entry_signal = (latest_priceh <  latest_priceh_20sma) and (latest_priceh_20sma <  latest_priceh_50sma) and  \
                              (latest_priceh <  latest_priceh_50sma) and (sma_slope_20hr and sma_slope_50hr) and \
                              macd_bullish_signal_1hr and (latest_pricem < latest_pricem_30sma )


      if below_price_threshold_ATR and latest_price > latest_price_5ema and below_price_threshold_ATR and refined_entry_signal:
        entry_signal = "Extended Short Entry"
      elif latest_price <= latest_price_8ema and above_price_threshold_ATR and refined_entry_signal :
        entry_signal = "Aline Entry"
      else:
        entry_signal = "Skip"
      results.append([ticker, entry_signal])
    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["Asset", "Entry_Signal"])
    return df_results

# Multi-timeframe strategy check returning a DataFrame
def check_mtf_entry(tickers):
    results = []

    for ticker in tickers:
        monthly_df = get_monthly_data(ticker)
        weekly_df = get_weekly_data(ticker)
        daily_df = get_daily_data(ticker)

        if is_monthly_trend_bullish(monthly_df) and is_weekly_trend_bullish(weekly_df):
            if  is_daily_entry_signal(daily_df):
                entry_signal = "Entry Confirmed ✅"
            else:
                entry_signal = "No Entry Yet on Daily Timeframe ⏳"
        else:
            entry_signal = "Monthly Trend Not Bullish ❌"

        results.append([ticker, entry_signal])
        time.sleep(2)  # Add a delay of 1 second between requests

    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["Asset", "Entry_Signal"])
    return df_results



In [184]:
# Multi-time frame entry Check
#df_results = pd.DataFrame(results).dropna()
etfs_to_check = df_o['Asset'].tolist()  #df_o['ETF'] #df_results['ETF'].tolist()

df_signals = check_mtf_entry(etfs_to_check)


df_signals.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Asset,Entry_Signal
0,THS,Entry Confirmed ✅
1,HNST,Monthly Trend Not Bullish ❌
2,BRCC,Monthly Trend Not Bullish ❌
3,BYND,Monthly Trend Not Bullish ❌
4,EPC,No Entry Yet on Daily Timeframe ⏳


## Generate Sell list

In [185]:
df_final = df_signals[df_signals['Entry_Signal'] =="Entry Confirmed ✅"]
final_etfs_to_check = df_final['Asset'].tolist()


sell_list = check_entry_conditions(final_etfs_to_check)


sell_list


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Asset,Entry_Signal
0,THS,Skip
1,SMPL,Skip
2,UTL,Aline Entry


In [186]:
# Apply TA filters and prioritize ETFs
results = []
sell_list = sell_list[sell_list['Entry_Signal'].isin(['Aline Entry'])]


for etf in sell_list['Asset'].to_list():
   df =get_daily_data(etf)
   price = df['Close'].iloc[-1].iloc[0]
   below_50sma = price  < df['50_day_SMA'].iloc[-1]
   below_20sma = price  < df['20_day_SMA'].iloc[-1]


   if below_50sma and below_20sma  :
    support_level, latest_price, trail,ema_level = calculate_risk_reward(df)
    entry_price = latest_price - max(0., 0.25*trail)
    trail = 1.5* trail
    risk = np.abs(support_level -entry_price)
    take_profit = entry_price - (1.5*risk)
    reward =  entry_price - take_profit
    support = support_level
    risk_reward_ratio = reward / risk
    # Ensure risk is greater than zero before division
    if risk > 0:

        rr_ratio  = reward / risk
    else:
        rr_ratio = np.nan

    stop_loss_perc = ((entry_price-support_level )/entry_price )*100
    take_profit_perc = (( entry_price - take_profit)/entry_price )*100
    # Fetch the Entry_Signal from buy_list
    entry_signal = sell_list.loc[sell_list['Asset'] == etf, 'Entry_Signal'].values[0]

    # Append results with Entry_Signal
    results.append({
            "Asset": etf,
            "Risk-Reward": rr_ratio,
            "Stop Loss": support_level,
            "Take Profit": take_profit,
            "Current Price": latest_price,
            "Entry Price": entry_price,
            "8EMA": ema_level,
            "Trail Price": trail,
            "Entry Signal": entry_signal,  # Add entry signal
            "stop_loss_perc": stop_loss_perc,
            "take_profit_perc": take_profit_perc
        })

    time.sleep(2)  # Add a delay of 1 second between requests


# Sort ETFs by highest risk-to-reward ratio
try:
   df_results = pd.DataFrame(results).dropna().sort_values(by="Risk-Reward", ascending=False).reset_index(drop = True)
except Exception as e:
  print("No Asset to buy today, check back some other time!")
  df_results = pd.DataFrame({"Asset": ["No Asset available"]})

df2 = df_results.merge(df_o[['Asset','Type', 'score']], on='Asset', how='left')
df2['timestamp'] = datetime.now()
df2 = df2.sort_values(by='score', ascending=True)
df2.head()

[*********************100%***********************]  1 of 1 completed


,Asset,Risk-Reward,Stop Loss,Take Profit,Current Price,Entry Price,8EMA,Trail Price,Entry Signal,stop_loss_perc,take_profit_perc,Type,score,timestamp
0,UTL,2.0,50.304635,37.9004,46.59,46.16989,47.422785,2.520661,Aline Entry,-8.955501,17.911002,Stock,-0.68,2025-08-27 04:57:08.525364


# Generate short list